In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_wlp_wls as shared
import scipy.integrate
from halotools.empirical_models import PrebuiltHodModelFactory

In [ ]:
SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
    import params as pa
elif (SURVEY=='LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

Goal: compare our $<N_{\rm sat}>$ vs $M_h$ and $<N_{\rm cen}>$ vs $M_h$ for Zu & Mandelbaum 2015 to the PreBuilt model in HaloTools

In [ ]:
# Define a halo mass range
#lgMh = np.linspace(9.5, 15.5, 100)
#Mh = 10**(lgMh)

In [ ]:
# Jan 8: implementing the fact that there is a typo in equation 19 of Zu and Mandelbaum (10^ vs exp)
# Check how old and new relationships look for sanity check
Ms = np.logspace(8, 12, 5000)
m = Ms / pa.Mso
Mh_old = pa.M1 * m**(pa.beta) * np.exp( m**pa.delta / (1. + m**(-pa.gamma)) - 0.5)
Mh_new = pa.M1 * m**(pa.beta) * 10**( m**pa.delta / (1. + m**(-pa.gamma)) - 0.5)

In [ ]:
plt.figure()
plt.loglog(Ms, Mh_old, 'b', label='old')
plt.hold(True)
plt.loglog(Ms, Mh_new, 'm', label='new')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.loglog(Mh_old, Ms, 'b', label='old')
plt.hold(True)
plt.loglog(Mh_new, Ms, 'm', label='new')
plt.xlim(10**10, 10**16)
plt.legend()
plt.show()

In [ ]:
bracket_term = m**pa.delta / (1. + m**(-pa.gamma)) - 0.5

plt.figure()
plt.semilogx(Ms, bracket_term)
plt.show()

For SDSS we get $M_*^{\rm low} = 6.8 \times 10^{9}$

In [ ]:
Mslow = 6.8*10**9

Get $<N_{\rm sat}>$ and $<N_{\rm cen}>$ from our code.

In [ ]:
Nsat_us = shared.get_Nsat_Zu(Mh, Mslow, 'tot', 'SDSS')
Ncen_us = shared.get_Ncen_Zu(Mh, Mslow, 'SDSS')

Set up and get the same from halotools

In [ ]:
model = PrebuiltHodModelFactory('zu_mandelbaum15', threshold = np.log10(Mslow), prim_haloprop_key = 'halo_m200m')
Nsat_HT = model.mean_occupation_satellites(prim_haloprop=Mh)
Ncen_HT = model.mean_occupation_centrals(prim_haloprop=Mh)

Plot and compare

In [ ]:
plt.figure()
plt.semilogy(lgMh, Nsat_us, 'g', label='$N_{\\rm sat}$, me')
plt.hold(True)
plt.semilogy(lgMh, Nsat_HT, 'm', label='$N_{\\rm sat}$, HT')
plt.ylim(0.01, 10000)
plt.ylabel('$N_{\\rm sat}$')
plt.xlabel('$M_h / h$')
plt.legend(loc=2)
#plt.show()
plt.savefig('./plots/Nsat_me_vs_halotools_SDSSshapes.pdf')

plt.figure()
plt.plot(lgMh, Ncen_us, 'g', label='$N_{\\rm cen}$, me')
plt.hold(True)
plt.plot(lgMh, Ncen_HT, 'm', label='$N_{\\rm cen}$, HT')
#plt.ylim(0.01, 100)
plt.ylabel('$N_{\\rm cen}$')
plt.xlabel('$M_h / h$')
plt.legend(loc=2)
#plt.show()
plt.savefig('./plots/Ncen_me_vs_halotools_SDSSshapes.pdf')

These are two high compared to the figure. Also, some of them are overlapping. What's wrong here?
Let's try to reproduce the right hand side of Figure 8, the Stellar - Halo mass relation

In [ ]:
Msvec = np.logspace(np.log10(7*10**8), np.log10(6*10**11), 100)
#Mh_out = shared.fSHMR_inverse(Msvec, 'SDSS')

#plt.figure()
#plt.loglog(Mh_out, Msvec)
#plt.ylabel('$M_{*}$')
#plt.xlabel('$M_h$')
#plt.xlim(10**11, 3*10**15)
#plt.ylim(7*10**8, 6*10**11)
#plt.show()

This is wrong too. Okay, let's try explicitly typing out the relation as in the paper rather than calling the funciton.

It's because I'm not accounting for the scatter. There is actually a formula in Ying's paper which provides a fit to the correct thing that we CAN directly invert (I think), not accounting for scatter. Let's try this.

In [ ]:
Msvec = np.logspace(np.log10(2*10**8), np.log10(10**13), 100)
Mh_get = shared.Mh_atfixed_Ms(Msvec)
lgMh_fit = np.log10(Mh_get)

plt.figure()
plt.loglog(Msvec, 10**lgMh_fit)
plt.ylabel('$M_{h}$')
plt.xlabel('$M_*$')
plt.ylim(5*10**10, 3*10**15)
plt.xlim(2*10**8, 10**12)
plt.show()

Okay, but what we actually want is $M_*(M_h)$. We know that at a fixed halo mass, the star mass has a log-normal scatter, and we know this scatter. So we can get, for each $M_h$, a mean $lnM_*(Mh)$ (from the relation of equation 19) and a series of other $ln M_*$ values drawn from a lognormal distribution aroumd this. The average of these $ln(M_*)$ value is then the mean value to associate with M_h. Try this.

In [ ]:
# Define an extended stellar vector for integrating over
Msvec = np.logspace(np.log10(2*10**8), np.log10(10**13), 100)
lnMstar = np.log(Msvec)

# For each halo mass
lgMh = np.linspace(11.5, 14., 100)
Mh = 10**(lgMh)
fshmr = np.zeros(len(lgMh))
for mi in range(0,len(lgMh)):
    lnMstar_mean = np.log(shared.get_fSHMR(Mh[mi], 'SDSS'))
    siglnM = shared.get_sigMs(Mh[mi], 'SDSS')
    # Define the lognormal distribution of M*
    p = 1. / (np.sqrt(2 * np.pi) * siglnM) * np.exp(-(lnMstar - lnMstar_mean)**2 / (2. * siglnM**2))
    #norm = np.sum(p)
    #p_normed = p / norm
    # Now sample from this distribution a bunch of lnM* values
    #lnMslist= np.random.choice(np.log(Msvec), p=p_normed, size = 10000)
    # Now get their mean
    lnMs_avg = scipy.integrate.simps(lnMstar * p, lnMstar)
    #lnMs_avg = np.mean(lnMslist)
    fshmr[mi] = np.exp(lnMs_avg)
    

The issue I am having now is the following.

I know $f_{\rm SHMR}(M_h) = exp(<ln(M_*(M_h))>)$

I know that $M_*$ is lognormally distributed at fixed halo mass, and I know its scatter.

But, how do I get its mean? This seems like a circular argument. I am trying to integrate:
$\int p(M_*,M_h) ln(M_*) d ln(M_*)$ 
to get the mean value $<ln(M_*(M_h))>$. But to get $p(M_*, M_h)$ in the first place, I require this mean value. How do I get out of this circle?

In [ ]:
plt.figure()
plt.loglog(Mh, fshmr)
plt.ylabel('$M_{*}$')
plt.xlabel('$M_h$')
plt.xlim(10**11, 3*10**15)
plt.ylim(7*10**8, 6*10**11)
plt.show()